In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/RUSTEYE/dataset_cnn_v2.zip

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 80

In [3]:
train_dir = '/content/dataset_cnn_v2/train'
val_dir   = '/content/dataset_cnn_v2/valid'
test_dir = '/content/dataset_cnn_v2/test'

# Data augmentation cho train
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Train generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Validation generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 945 images belonging to 4 classes.
Found 202 images belonging to 4 classes.
Found 205 images belonging to 4 classes.


In [4]:
base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
for layer in base_model.layers:
    layer.trainable = False

# Thêm các layer phía trên
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [5]:
# Compile
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# Callbacks
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)

In [7]:
# Train
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stopping]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 63s 1s/step - accuracy: 0.3698 - loss: 1.4613 - val_accuracy: 0.5495 - val_loss: 0.9964
Epoch 2/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 22s 741ms/step - accuracy: 0.5252 - loss: 1.0189 - val_accuracy: 0.5891 - val_loss: 0.8815
Epoch 3/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 23s 771ms/step - accuracy: 0.5431 - loss: 0.9500 - val_accuracy: 0.5594 - val_loss: 0.8813
Epoch 4/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 23s 760ms/step - accuracy: 0.5968 - loss: 0.8709 - val_accuracy: 0.6188 - val_loss: 0.8373
Epoch 5/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 21s 683ms/step - accuracy: 0.5844 - loss: 0.8848 - val_accuracy: 0.6139 - val_loss: 0.8859
Epoch 6/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 21s 677ms/step - accuracy: 0.5525 - loss: 0.9126 - val_accuracy: 0.6535 - val_loss: 0.8457
Epoch 7/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 21s 697ms/step - accuracy: 0.6045 - loss: 0.8151 - val_accuracy: 0.6238 - val_loss: 0.8681
Epoch 8/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 42s 713ms/step - accuracy: 0.6000 - loss: 0.8793 - val_accurac

In [8]:
# Load best model
from tensorflow.keras.models import load_model
best_model = load_model('best_model.keras')

In [9]:
# Đánh giá trên test set
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

test_loss, test_accuracy = best_model.evaluate(test_generator)
print(f'\nTest Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Lấy nhãn class
class_labels = list(test_generator.class_indices.keys())
predictions = best_model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Metrics
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_labels, digits=4))

# Accuracy theo từng class
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_labels, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.4816 - loss: 0.9696

Test Loss: 0.8223
Test Accuracy: 62.44%
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step
Accuracy: 62.4390243902439
Precision: 69.95527145480892
Recall: 62.4390243902439
F1 Score: 58.29139837908327

Classification Report:
                precision    recall  f1-score   support

 bong-troc-son     0.8750    0.1373    0.2373        51
day-cap-ri-set     0.5269    0.8596    0.6533        57
        ri-set     0.5574    0.6538    0.6018        52
       vet-nut     0.8837    0.8444    0.8636        45

      accuracy                         0.6244       205
     macro avg     0.7107    0.6238    0.5890       205
  weighted avg     0.6996    0.6244    0.5829       205


Accuracy for each class:
bong-troc-son: 0.14
day-cap-ri-set: 0.86
ri-set: 0.65
vet-nut: 0.84
